In [141]:
# Before run this note please install follow packages
# pip3 install pandas pyarrow pandasql

# Q1
import pandas as pd
converters = {
    'lpep_pickup_datetime': lambda x : pd.to_datetime(x, format='%Y%m%d %H:%M:%S'),
    'Lpep_dropoff_datetime': lambda x : pd.to_datetime(x, format='%Y%m%d %H:%M:%S'),
    'Store_and_fwd_flag': lambda x : False if x == None else x == 'Y',
    'Ehail_fee': lambda x : 0 if not x else x,
    'Trip_type ': lambda x : -1 if not x else int(x)
}
df = pd.read_csv('green_tripdata_2013-09.csv', skiprows=[1,1], converters=converters).rename(columns=lambda x: x.strip().lower())
df.to_parquet('/tmp/green_tripdata_2013-09.parquet')
print("Converted!")

#Q2
import pandasql as ps
# JFK airport location
# Latitude: 40.6398262
# Longitude: -73.7787443 
minLat = 40.624299
maxLat = 41.675766
minLon = -74.803331
maxLon = -73.770735

bounding_box_query = """
case
when Pickup_longitude >= %(minLon)s and Pickup_longitude <= %(maxLon)s and Pickup_latitude >= %(minLat)s and Pickup_latitude <= %(maxLat)s then 1
when Dropoff_longitude >= %(minLon)s and Dropoff_longitude <= %(maxLon)s and Dropoff_latitude >= %(minLat)s and Dropoff_latitude <= %(maxLat)s then 2
else 0
end in_jfk
""" % {"minLon": minLon, "maxLon": maxLon, "minLat": minLat, "maxLat": maxLat}

query = """
SELECT df.*,
strftime('%H', lpep_pickup_datetime) as hour,
strftime('%w', lpep_pickup_datetime) as dow,
(strftime('%s', Lpep_dropoff_datetime) - strftime('%s', lpep_pickup_datetime)) as seconds_duration,
""" + bounding_box_query + " FROM df"

df1 = ps.sqldf(query, locals())
df1.to_parquet('/tmp/green_tripdata_2013-09_and_addition.parquet')
print("Done!")


Converted!
       vendorid        lpep_pickup_datetime       lpep_dropoff_datetime  \
0             2  2013-09-01 00:02:00.000000  2013-09-01 00:54:51.000000   
1             2  2013-09-01 00:02:34.000000  2013-09-01 00:20:59.000000   
2             2  2013-09-01 00:03:06.000000  2013-09-01 00:28:03.000000   
3             2  2013-09-01 00:03:30.000000  2013-09-01 00:23:02.000000   
4             2  2013-09-01 00:05:12.000000  2013-09-01 00:30:55.000000   
...         ...                         ...                         ...   
49642         2  2013-09-30 23:57:52.000000  2013-10-01 00:01:40.000000   
49643         1  2013-09-30 23:57:57.000000  2013-10-01 00:08:56.000000   
49644         2  2013-09-30 23:58:40.000000  2013-10-01 00:00:46.000000   
49645         2  2013-09-30 23:59:41.000000  2013-10-01 00:12:10.000000   
49646         2  2013-09-30 23:59:47.000000  2013-10-01 00:06:15.000000   

       store_and_fwd_flag  ratecodeid  pickup_longitude  pickup_latitude  \
0           